Введение:

Необходимо найти решение определения уровня сложности английского языка в фильмах по субтитрам.

Заказчик предоставил данные:

- английские словари Oxford, в которых слова распределены по уровню сложности
- набор файлов-субтитров, рассортированных по каталогам в соответствии с уровнем сложности
- excel файл со список несортированных фильмов и указанием их уровня

Цель:

- создание модели определения уровня сложности

Задачи:

- изучение и обработка предоставленного материала
- обработка текста субтитров и подготовка для машинного обучения
- расчет и добавление дополнительных признаков
- тестирование модели и подбор гиперпараметров
- выделение наиболее удачных признаков
- обучение оптимальной модели на датасете с наиболее полезными признаками



In [1]:
import os
import re
import warnings

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from joblib import dump, load

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from sklearn.cluster import KMeans

try:
    import spacy
except:
    !pip install spacy
    !python -m spacy download en
    import spacy

try:
    import chardet
except:
    !pip install chardet
    import chardet

try:
    from pypdf import PdfReader
except:
    !pip install pypdf
    from pypdf import PdfReader

try:
    import pysrt
except:
    !pip install pysrt
    import pysrt

In [2]:
warnings.filterwarnings("ignore")
plt.style.use('ggplot')
nlp = spacy.load("en_core_web_sm")

SCORES_PATH    = '/Users/keybl/Desktop/'
SUBTITLES_PATH = '/Users/keybl/Desktop/Subtitles_all'
OXFORD_PATH    = '/Users/keybl/Desktop/English_level/Oxford_CEFR_level'

ENGLISH_LEVELS = ['A1', 'A2', 'B1', 'B2', 'C1', 'C2']

RANDOM_STATE = 7

Загрузим имеющиеся у нас словари
- прочитаем их 
- разобьем на строки,
- выделим слова,
- определим уровень
- сохраним, из какого словаря взяли слово
- удалим дубликаты, оставим те случаи, где слову присвоена более высокая сложность

In [3]:
def process_pdf(content):
    
    current_level = ''
    words_by_level = {key:[] for key in ENGLISH_LEVELS}
    
    for line in content:
        line = line.strip()

        if 'Oxford' in line or 'English' in line:
            pass
        
        elif line in ENGLISH_LEVELS:
            level = line
            
        elif ' ' in line:
            line = re.sub(r'\d|,', '', line.lower())   # remove digits and commas and transform to lowercase
            words_by_level[level] += [line.split()[0]] #get the first occurency in line as word
            
        else:
            pass
    
    return words_by_level


content = []
oxford_words = pd.DataFrame(columns=['word', 'level', 'source', 'type'])

# load all pdfs
for dirname, _, filenames in os.walk(OXFORD_PATH):
    for filename in filenames:
        print(filename)
        reader = PdfReader(f'{OXFORD_PATH}/{filename}')
        for page in reader.pages:
            content += page.extract_text().splitlines()
        processed = process_pdf(content)
        for level in processed:
            dict_type = 'us' if 'American' in filename else 'uk'
            oxford_words = pd.concat([oxford_words,
                                      pd.DataFrame({'word'   : processed[level],
                                                    'level'  : level,
                                                    'source' : filename,
                                                    'type'   : dict_type
                                                   })
                                     ])

# sort words and keep unique with higher levels
oxford_words = oxford_words.sort_values(by=['word', 'level'], ascending=True)
oxford_words = oxford_words.drop_duplicates(subset=['word'], keep='last')

# после сортировки и удаления дубликатов остался только американский инглиш
# похоже там все уровни слов считаются выше

oxford_words.groupby('level') \
            .agg({'word':'count', 'source': 'unique'}) \
            .style.format({'word':'{:,.0f} слов'})

American_Oxford_3000_by_CEFR_level.pdf
The_Oxford_3000_by_CEFR_level.pdf
The_Oxford_5000_by_CEFR_level.pdf
American_Oxford_5000_by_CEFR_level.pdf


,word,source
level,,
A1,735 слов,['American_Oxford_5000_by_CEFR_level.pdf']
A2,747 слов,['American_Oxford_5000_by_CEFR_level.pdf']
B1,758 слов,['American_Oxford_5000_by_CEFR_level.pdf']
B2,"1,446 слов",['American_Oxford_5000_by_CEFR_level.pdf']
C1,"1,198 слов",['American_Oxford_5000_by_CEFR_level.pdf']


In [4]:
dump(oxford_words,'english_level_oxford.joblib')

['english_level_oxford.joblib']

In [5]:
# process line
def process_line(line):
    if re.search(r'[A-Za-z]',line): 
        line = line.lower()
        line = re.sub(r'\n', ' ', line)                            # remove new lines
        line = re.sub(r'- ', ' ', line)                            # remove dash
        line = re.sub(r'\<[^\<]+?\>', '', line)                    # remove html tags
        line = re.sub(r'\([^\(]+?\)', '', line)                    # remove () parenthesis
        line = re.sub(r'\[[^\[]+?\]', '', line)                    # remove [] parenthesis
        line = re.sub(r'^([\w#\s]+\:)', ' ', line)                 # remove speaker tag
        line = re.sub(r'[^[:alnum:][:punct:][:blank:]]',' ', line) # remove all other non-speach shars
        line = re.sub(r'\s\s+', ' ', line).strip()                 # remove extra spaces
    return line


# process text line by line   
def process_text(content):
    text = []
    duration  = []
    charsrate = []
    wordsrate  = []
    for item in content:
        if not hasattr(item, 'duration'):
            print('no')
        if item.duration.ordinal>0:
            line = process_line(item.text_without_tags)
            text.append(line)
            duration.append(item.duration.ordinal/1000)
            charsrate.append(item.characters_per_second)
            wordsrate.append(len(line.split())/item.duration.ordinal*1000.0)
    return ' '.join(text), duration, charsrate, wordsrate


# process file
def process_srt(dirname, filename):
    global count
    if not filename.endswith('.srt'):                        # skip non srt files
        print('посторонний файл', filename)
        return False
    fullpath = os.path.join(dirname,filename)
    try:
        enc = chardet.detect(open(fullpath, "rb").read())['encoding']
        content = pysrt.open(fullpath, encoding=enc)
    except:
        print('не прочиталось', filename)
        return False
    return process_text(content)                            # clean text and return


# movies dataset template
movies = pd.DataFrame(columns=['filename', 
                               'content', 
                               'duration', 
                               'charsrate', 
                               'wordsrate', 
                               'level', 
                               'lemmas'] + ENGLISH_LEVELS + [l+'ratio' for l in ENGLISH_LEVELS]
                     )

# recursive walk through dirs
for dirname, _, filenames in os.walk(SUBTITLES_PATH):
    for filename in filenames:
        level  = dirname.split('/')[-1]                   # get level name from dir
        result = process_srt(dirname, filename)           # process file
        if result:                                        # add movie to dataframe
            subs, duration, charsrate, wordsrate = result
            movies.loc[len(movies)] = \
                {'filename' : filename.replace('.srt', ''),
                 'content'  : subs,
                 'duration' : duration, 
                 'charsrate': charsrate, 
                 'wordsrate': wordsrate, 
                 'level'    : level
                }

movies.head(3)

посторонний файл .DS_Store


,filename,content,duration,charsrate,wordsrate,level,lemmas,A1,A2,B1,B2,C1,C2,A1ratio,A2ratio,B1ratio,B2ratio,C1ratio,C2ratio
0,The IT Crowd - 1e06-Aunt Irma Visits,all right! ok! all right! yes! ok! whoa! i kno...,"[2.002, 2.128, 1.809, 3.129, 3.482, 3.447, 3.0...","[6.993006993006993, 8.928571428571429, 2.76395...","[1.4985014985014986, 1.8796992481203008, 0.552...",Subtitles_all,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Chip S1 E4,♪ sometimes some crimes ♪ ♪ go slippin' throug...,"[3.916, 2.457, 3.541, 2.832, 1.999, 1.624, 2.2...","[6.384065372829418, 13.431013431013431, 7.3425...","[1.2768130745658834, 2.849002849002849, 1.6944...",Subtitles_all,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Dredd(2012),america is an irradiated wasteiand. within it ...,"[3.086, 1.918, 2.919, 1.751, 3.336, 3.336, 2.6...","[11.34154244977317, 11.470281543274245, 12.332...","[1.6202203499675956, 2.6068821689259645, 2.055...",Subtitles_all,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Excel labels processing

Часть файлов лежит в общем каталоге без указания уровня сложности. Для них заказчик предоставил excel-файл со списком фильмов и уровнями.

Загрузим файл и изучим его:

- удалим дубликаты
- поправим несовпадения названий
- из указанных уровней оставим более сложный вариант
- удалим фильмы, для которых нет информации по уровню сложности

In [25]:
DATASETS_PATH_LOCAL = '/Users/keybl/Desktop/'                               # local path to data
DATASETS_PATH_REMOTE = '/Users/keybl/Desktop/'                             # remote path to data

SUBTITLES_PATH_LOCAL = '/Users/keybl/Desktop/subtitles_all/'                    # local path to data (subtitles)
SUBTITLES_PATH_REMOTE = '/Users/keybl/Desktop/subtitles_all/'    # remote path to data (subtitles)

CR = '\n'                                                       # new line char sequence

In [17]:
def custom_read_csv(file_name, separator=','):
    """
    reading dataset of .csv format:
       first from local storage;
       if unsuccessful from remote storage.
    """

    path_local = f'{PATH_LOCAL}{file_name}'
    path_remote = f'{PATH_REMOTE}{file_name}'
    
    if os.path.exists(path_local):
        return pd.read_csv(path_local, sep=separator)

    elif os.path.exists(path_remote):
        return pd.read_csv(path_remote, sep=separator)

    else:
        print(f'File "{file_name}" not found at the specified path ')

In [18]:
# text styles
class f:
    BOLD = "\033[1m"
    ITALIC = "\033[3m"
    END = "\033[0m"

In [19]:
# Pandas defaults
pd.options.display.max_colwidth = 100
pd.options.display.max_rows = 500
pd.options.display.max_columns = 100
pd.options.display.float_format = '{:.3f}'.format
pd.options.display.colheader_justify = 'left'

In [20]:
# others
warnings.filterwarnings('ignore')

Подготовка слов из субтитров к фильмам

Функция для чтения субтитров из файла

In [21]:
def read_subtitles(file_name, file_ext='srt', file_encoding='latin-1'):
    """
    Чтение текста из файла file_name.
    Возвращает датафрейм с текстом и статус операции:
        'OK' – файл найден;
        'no subtitles' – файл с субтитрами не найден.
    """

    path_local = f'{SUBTITLES_PATH_LOCAL}{file_name}.{file_ext}'
#     print(f'file {path_local} in process')
    
    if os.path.exists(path_local):
        return pd.read_table(path_local, names=['subs'], encoding=file_encoding), 'OK'

    else:
        print(f'file {f.BOLD}{path_local}{f.END} not found')
        return pd.DataFrame(), 'no subtitles'

Функция для очистки текста

In [22]:
def clean_text(text):
    '''
    Принятый текст в типа string:
        - переводит в нижний регистр;
        - заменяет на пробел символ перевода строки;
        - заменяет на пробел теги (текст, заключенный в квадратные или фигурные скобки);
        - заменяет на пробел все символы, кроме букв, дефисов и апострофов внутри слов;
        - удаляет все слова из 1-2 букв;
        - заменяет цепочки пробелов на одинарный пробел.
    Возвращает очищенный текст.
    '''
    # приведение значений к нижнему регистру
    text = text.lower()
    
    # удаление перевода строки
    text = text.replace('\n', ' ')
    
    # удаление тегов: частей строк между символами < и >, { и }
    text = re.sub('<.*?>', ' ', text)
    text = re.sub('{.*?}', ' ', text)

    #-------------------------------------------------------------------------------------------------
    # удаление всех символов, кроме букв, пробелов и внутренних дефисов и апострофов
    
    # замена дефисов внутри слов на временный заполнитель (перед удалением небуквенных символов)
    text = re.sub('([a-zA-Z])-([a-zA-Z])', '\\1temporarydefis\\2', text)
    # замена апострофов внутри слов на временный заполнитель (перед удалением небуквенных символов)
    text = re.sub("([a-zA-Z])'([a-zA-Z])", '\\1temporaryapostrof\\2', text)
    # удаление всех символов, кроме букв и пробелов
    text = re.sub('[^a-zA-Z\s]', ' ', text)
    # восстановление внутренних дефисов (замена временного заполнителя на настоящий дефис)
    text = text.replace('temporarydefis', '-')
    # восстановление внутренних апострофов (замена временного заполнителя на настоящий апостроф)
    text = text.replace('temporaryapostrof', "'")
    #-------------------------------------------------------------------------------------------------

    # удаление всех символов, кроме букв и пробелов
#     text = re.sub('[^a-zA-Z\s]', ' ', text)                      # на случай, если дефисы и апострофы тоже нужно удалить
    
    # удаление слов из одной-двух букв (за редким исключением)
#     subtitles = re.sub(r'\s\b\w{1,2}\b\s', ' ', subtitles)       # слово окружено пробелами
    text = re.sub(r'\b\w{1,2}\b', ' ', text)                       # неважно, чем окружено слово
    
    # замена нескольких последовательных пробелов на одинарные
    text = re.sub(' +', ' ', text)
    
    # удаление крайних пробелов (мелочь, но пусть будет)
    text = text.strip()

    return text

Функция для подготовки субтитров

In [23]:
def prepare_subtitles(movie_name, clean=True):
    '''
    Загружает субтитры для указанного фильма movie_name с помощью функции read_subtitles.
    Выполняет очистку текста от лишней информации (служебные данные, знаки препинания и т.п.) с помощью функции clean_text.
    Возвращает очищенный текст.
    '''
    
    # чтение файла с субтитрами; имя файла совпадает с названием фильма в таблице фильмов
    df, status = read_subtitles(movie_name)
    if status == 'no subtitles':
        return 'no subtitles'
    
    # удаление служебных строк (начинаются с цифры)
    df = df[~df.subs.str.match('^\d')].reset_index(drop=True)
    
    # соединение отдельных субтитров в единый корпус
    subtitles = df.subs.str.cat(sep=' ')
    
    if clean:
        subtitles = clean_text(subtitles)
    
    return subtitles

Чтение таблицы с фильмами

In [26]:
data = pd.read_excel(DATASETS_PATH_LOCAL + 'movies_labels.xlsx', index_col='id').reset_index(drop=True)

In [27]:
data.sample(3)

,Movie,Level
206,Downton Abbey - S01E06 - Episode 6.eng.SDH,C1
307,The.Umbrella.Academy.S01E04.WEBRip.x264-ION10,A2
360,Interstellar-2014-HDCAM-NEW-SOURCE-READNFO-XVID-AC3-ACAB,B2


Добавление слов из субтитров в таблицу фильмов

In [28]:
data['Subtitles'] = data.Movie.apply(prepare_subtitles, clean=True)

file /Users/keybl/Desktop/subtitles_all/Glass Onion.srt not found
file /Users/keybl/Desktop/subtitles_all/Matilda(2022).srt not found


Для некоторых фильмов файл с субтитрами не найден. Однозначно определить, что это за фильм, и скачать для него субтитры не удалось.

Удаление из таблицы фильмов без субтитров

In [30]:
data = data[data.Subtitles != 'no subtitles']

Пример для ручной проверки

Очищенные субтитры

In [31]:
film_name = 'The.Man.Called.Flintstone.1966.1080p.WEB-DL.DD+2.0.H.264-DAWN'
prepare_subtitles(film_name, clean=True)

"who they call when the chips are startin fall just when the villains are winning the trouble' beginning brew explosion the man called flintstone that' who who' always there there rescue some maiden fair man with instincts unerring full the old daring- the man called flintstone that' who thrives diet danger and ' stranger the unjust with romance who the man the man called flintstone rumbling screeches don' let him get away don' worry won' hee hee hee hee hee hee hee hee hee angry muttering detour watch out snores angry muttering have parachute too good let' follow him that our parachute hee hee hee hee hee hee hee hee hee some parachute shut you angry muttering crash have him trapped good let' squish him ali hurry hurry must not escape won' ' got him now panting giggles chuckles both laughing that' the end rock slag nice work bobo thank you ali chuckles they think they' finished off they don' know how tough secret agents are this secret agent rock slag reporting chief boulder come chie

Корректировка Level

Для фильмов с несколькими категориями нужно оставить одну (самую высокую).
Заменить категории с плюсом на обычные, поскольку плюсовых категорий в CEFR нет.

In [32]:
data.Level = data.Level.apply(lambda Level: Level.replace('+', '')[-2:])

In [34]:
data.to_csv(f'/Users/keybl/Desktop/EDA_movies_subtitles.csv', index=False)

Предобработку субтитров можно/нужно добавить в пайплайн модели.

Если не используются новые сгенерированные признаки, модель работает на новых данных и без предварительной обработки.

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 514 entries, 0 to 515
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Movie      514 non-null    object
 1   Level      514 non-null    object
 2   Subtitles  514 non-null    object
dtypes: object(3)
memory usage: 16.1+ KB


In [36]:
data.sample(5)

,Movie,Level,Subtitles
302,The.Hollow.S02E09.Fire.720p.NF.WEB-DL.DDP5.1.H.264-NTb,A2,what are you looking ' looking for the perfect target bingo then ' huff and ' puff and ' blow yo...
128,Seven.Worlds.One.Planet.S01E06.2160p.BluRay.Remux.eng,B1,one continent our planet changes more dramatically than any other north america whole landscapes...
142,Crazy4TV.com - Suits.S06E08.720p.BluRay.x265.HEVC.Crazy4ad,B2,harvey reading your execution date has been set when thirty days you said you were gonna find ma...
161,Suits.Episode 15- Tick Tock,B2,previously suits don' owe harvey shit may have gotten out but done came and put there the first ...
251,13.Reasons.Why.S01E11.720p.WEBRiP.x265.ShAaNiG,A2,sync and corrections www addic com hannah ' told you about two the worst decisions ever made and...


Употребление слов в субтитрах из словаря Oxford

Будет исследовано распределение слов разных уровней в фильмах разных уровней.

Будет учтено как общее количество употреблений слова (даже если оно повторяется неоднократно), так и распределение уникальныхх слов (каждое слово будет учтено только один раз).

Идеальным резултатом будет выраженная зависимость между словами и фильмамит определенного уровня.

In [38]:
data_words = oxford_words

In [41]:
data_words.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4884 entries, 887 to 1432
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   word    4884 non-null   object
 1   level   4884 non-null   object
 2   source  4884 non-null   object
 3   type    4884 non-null   object
dtypes: object(4)
memory usage: 190.8+ KB


In [42]:
# группировка слов по уровням и преобразование в списки

series_oxford = data_words.groupby('level').word.agg(list)
series_oxford

level
A1    [a, about, above, across, action, activity, actor, actress, add, advice, afraid, afternoon, agai...
A2    [ability, able, accept, accident, according, achieve, active, actually, adult, advantage, advent...
B1    [absolutely, access, achievement, act, ad, addition, admire, admit, advanced, advise, afford, ag...
B2    [abandon, abroad, absolute, absorb, abstract, academic, accent, acceptable, accidentally, accomm...
C1    [abolish, abortion, absence, absent, absurd, abundance, abuse, academy, accelerate, acceptance, ...
Name: word, dtype: object

In [43]:
# преобразование текста с обработанными субтитрами в список, где каждый элемент – слово

data['subs_words'] = data.Subtitles.apply(lambda x: x.split())

In [44]:
def oxford_count(df, level):
    '''
    df: датафрейм для обработки
    level: уровень, соответствующий "American Oxford by CEFR level"
    
    Добавляет в датафрейм:
        поле вида 'level_sum' с количеством употреблений слов соответствующего уровня;
        поле вида 'level_unq' с количеством уникальных слов соответствующего уровня.
    '''
    
    subs = df['subs_words']
    oxford = series_oxford[level]
    
    df[f'{level}_sum'] = df['subs_words'].apply(lambda subs: sum(i in oxford for i in subs))
    df[f'{level}_unq'] = df['subs_words'].apply(lambda subs: sum(i in subs for i in oxford))
    
    return df

In [46]:
import re
from tqdm import tqdm

In [47]:
# список всех уровней CEFR, имеющихся в списке фильмов
level_list = sorted(list(data.Level.unique()))


# для каждого возможного уровня добавить в датафрейм:
# количество употреблений слов соответствующего уровня и
# количество уникальных слов соответствующего уровня

for level in tqdm(level_list):
    data = oxford_count(data, level)

100%|█████████████████████████████████████████████| 5/5 [02:50<00:00, 34.17s/it]


In [49]:
# общее количество употреблений слов и уникальных слов всех уровней

data['TOTAL_sum'] = 0
data['TOTAL_unq'] = 0

for level in level_list:
    data['TOTAL_sum'] += data[f'{level}_sum']
    data['TOTAL_unq'] += data[f'{level}_unq']


# подсчет доли разных уровней для употреблений слов и уникальных слов

for level in level_list:
    data[f'{level}_sum_%'] = data[f'{level}_sum'] / data['TOTAL_sum']
    data[f'{level}_unq_%'] = data[f'{level}_unq'] / data['TOTAL_unq']

In [50]:
# удаление ненужных полей

data = data.drop(['subs_words','TOTAL_sum','TOTAL_unq'], axis=1)

for level in level_list:
    data = data.drop(f'{level}_sum', axis=1)     # количество употреблений слов определенного уровня
    data = data.drop(f'{level}_unq', axis=1)     # количество уникальных слов определенного уровня    

In [51]:
# сводная таблица для оценки доли употреблений слов и уникальных слов разных уровней
# в зависимости от уровня фильма, установленного экспертом

data.groupby('Level').mean()

,A1_sum_%,A1_unq_%,A2_sum_%,A2_unq_%,B1_sum_%,B1_unq_%,B2_sum_%,B2_unq_%,C1_sum_%,C1_unq_%
Level,,,,,,,,,,
A1,0.669,0.585,0.141,0.189,0.104,0.113,0.058,0.085,0.027,0.028
A2,0.614,0.444,0.153,0.229,0.115,0.142,0.081,0.141,0.036,0.044
B1,0.616,0.404,0.151,0.229,0.115,0.155,0.083,0.161,0.034,0.051
B2,0.622,0.397,0.143,0.226,0.118,0.157,0.083,0.165,0.034,0.055
C1,0.619,0.390,0.147,0.226,0.116,0.161,0.083,0.167,0.035,0.055


Пояснение к названиям столбцов:
A1_sum_% – доля употребленных слов уровня A1 среди всех слов с известным уровнем.
A1_unq_% – доля уникальных слов уровня A1 среди всех слов с известным уровнем.
Аналогично и для других уровней.

Если в полученной таблице изучить столбцы сверху вниз, то видно, что сколько либо заметно отличается только уровень A1, как для уникальных слов из словаря Oxford, так и для общего количества их употреблений.
Для остальных уровней доля слов разных уровней примерно одинакова.

In [53]:
#data.to_csv(f'/Users/keybl/Desktop/EDA_movies_subtitles.csv', index=False)

In [54]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 514 entries, 0 to 515
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Movie      514 non-null    object 
 1   Level      514 non-null    object 
 2   Subtitles  514 non-null    object 
 3   A1_sum_%   514 non-null    float64
 4   A1_unq_%   514 non-null    float64
 5   A2_sum_%   514 non-null    float64
 6   A2_unq_%   514 non-null    float64
 7   B1_sum_%   514 non-null    float64
 8   B1_unq_%   514 non-null    float64
 9   B2_sum_%   514 non-null    float64
 10  B2_unq_%   514 non-null    float64
 11  C1_sum_%   514 non-null    float64
 12  C1_unq_%   514 non-null    float64
dtypes: float64(10), object(3)
memory usage: 56.2+ KB


In [55]:
data.sample(5)

,Movie,Level,Subtitles,A1_sum_%,A1_unq_%,A2_sum_%,A2_unq_%,B1_sum_%,B1_unq_%,B2_sum_%,B2_unq_%,C1_sum_%,C1_unq_%
162,Suits.Episode 16- 25th Hour,B2,previously suits has ruined clifford' life and you had been our lawyer never would have been con...,0.633,0.402,0.135,0.242,0.122,0.148,0.082,0.161,0.028,0.048
310,The.Umbrella.Academy.S01E07.WEBRip.x264-ION10,A2,the seventh hour the first day october woman went into labor jenkins meet your son this was unus...,0.623,0.442,0.141,0.223,0.107,0.136,0.085,0.154,0.044,0.046
323,The.Umbrella.Academy.S02E10.WEBRip.Netflix.en,A2,your father ready give the eulogy children the world full injustice good people die along with t...,0.617,0.397,0.153,0.257,0.110,0.124,0.091,0.169,0.029,0.054
385,17_Extra_English_-_Cyber_Stress.Eng_Syned,A1,this the story bridget and annie who share flat london and the boys next door nick and his frien...,0.684,0.606,0.109,0.158,0.106,0.118,0.072,0.089,0.029,0.030
22,Cinderella(1950),B1,cinderella you' lovely your name cinderella you' sunset frame though you' dressed rags you wear ...,0.597,0.424,0.156,0.223,0.125,0.159,0.078,0.147,0.044,0.047
